# Sampling from the von Mises - Fisher distribution

The von Mises - Fisher distribution is a probability distribution on the surface of a unit hypersphere of N dimensions: 

<div><center>
    $f_N(x ; \mu, \kappa)$
<center></div>

Where:
* $\mu$ is the mean direction 
* $\kappa$ is the concentration parameter

Wikipedia page: https://en.wikipedia.org/wiki/Von_Mises%E2%80%93Fisher_distribution 

Code used in this example is based on [this](https://stats.stackexchange.com/questions/156729/sampling-from-von-mises-fisher-distribution-in-python) and [this](https://github.com/clara-labs/spherecluster/blob/develop/spherecluster/util.py).

In [2]:
%matplotlib inline
import numpy as np
from sphere import *

## For ${\rm I\!R}^3$

For N = 3 dimensions, the von Mises - Fisher distribution is defined on the surface of a sphere.

In [3]:
# specify mu as a some 3D vector
mu = [2, -2, 3]

# normalise to a unit vector 
norm = np.linalg.norm(mu)
mu = mu / norm
print ('average direction:', mu)

# sampling
num_samples = 500
kappa = [1, 10, 100]

average direction: [ 0.48507125 -0.48507125  0.72760688]


In [4]:
# sample from vMF for a range of kappa
samples = []
for k in kappa:
    samples.append(Sphere().sample(num_samples, distribution = 'vMF', 
                                   mu = mu, kappa = k))

In [ ]:
# plot points on the unit sphere
Sphere().plot(data = samples)

## Sampling uniformly over the sphere

To contrast, consider a sampler which draws points uniformly on the spherical shell. See [here](http://mathworld.wolfram.com/SpherePointPicking.html) for different implementations.

In [ ]:
# sample uniformly over a unit sphere
samples = Sphere().sample(num_samples)
Sphere().plot(data = samples)

## Simulation in Stan

Using the vMF_rng defined in `vMF.stan`.

In [ ]:
import pystan
simulation = pystan.StanModel(file = 'vMF_sim.stan')

In [ ]:
# Use stan to simulate draws from the vMF
kappa = [1, 10, 100]
samples = []
for k in kappa:
    simu_parameters = {'N' : num_samples,
                       'mu' : mu,
                       'kappa' : k}

    simu = simulation.sampling(data = simu_parameters, algorithm = "Fixed_param", 
                               iter = 1, chains = 1, seed = 19920715)

    if(k == 1):
        data_sample = simu.extract(['vMF_samples'])['vMF_samples'][0]
    sample = simu.extract(['vMF_samples'])['vMF_samples'][0]
    samples.append(vMFSample(sample, k))

In [ ]:
Sphere().plot(data = samples)

## Fitting the data in Stan

Use a simple model to infer $\mu$ and $\kappa$ from vMF samples. The model is defined in `vMF_model.stan` and uses the vMF_lpdf defined in `vMF.stan`.

In [ ]:
model = pystan.StanModel(file = 'vMF_model.stan')

In [ ]:
sample = Sphere().sample(num_samples, distribution = 'vMF', mu = mu, kappa = 20).sample
data = {'N' : len(sample), 'd' : sample}
fit = model.sampling(data = data, seed = 19920715)

In [ ]:
print(fit)

# compare with true values
print('\ntrue mu:', mu)
print('true kappa:', 50)

In [ ]:
fig = fit.plot();
fig.set_size_inches(20, 16)
fig.tight_layout()